In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import squarify
import time
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
start = time.time()
df_Nov = pd.read_csv("./data/2019-Nov.csv")
df_Nov['event_time'] = pd.to_datetime(df_Nov['event_time'], format='%Y-%m-%d %H:%M:%S %Z')
end = time.time()
print(end - start)

## [RQ1]
A marketing funnel describes your customer’s journey with your e-commerce. It may involve different stages, beginning when someone learns about your business, when he/she visits your website for the first time, to the purchasing stage, marketing
funnels map routes to conversion and beyond. Suppose your funnel involves just three simple steps: 1) view, 2) cart, 3) purchase. Which is the rate of complete funnels?

# 1.a
What’s the operation users repeat more on average within a session? Produce a plot that shows the average number of times users perform each operation (view/removefromchart etc etc).

### 1.b
How many times, on average, a user views a product before adding it to the cart?

### 1.c
What’s the probability that products added once to the cart are effectively bought?

### 1.d
What’s the average time an item stays in the cart before being removed?

# 1.e
How much time passes on average between the first view time and a purchase/addition to cart?

In [ ]:
def view_purch_avg_time(df):

    df.loc[:, 'action'] = ''
    df.loc[df.event_type == 'view', 'action'] = 'view'
    df.loc[df.event_type.isin(['cart', 'purchase']), 'action'] = 'cart_purchase'

    def view_purch_timediff(x):
        if x.shape[0] == 1:
            return None
        return max(x) - min(x)


    df_first_groups = df.groupby(['product_id', 'user_id', 'action']).aggregate(time_first_action=pd.NamedAgg(
            column='event_time',
            aggfunc='min'
    )).reset_index()

    df_second_groups = df_first_groups.groupby(['product_id', 'user_id']).aggregate(time_difference=pd.NamedAgg(
            column='time_first_action', 
            aggfunc=view_purch_timediff
    )
    ).reset_index()
    
    return df_second_groups[pd.notnull(df_second_groups)['time_difference']]['time_difference'].mean()

## [RQ3]
For each category, what’s the brand whose prices are higher on average?

### 3.a
Write a function that asks the user a category in input and returns a plot indicating the average price of the products sold by the brand.

In [ ]:
def avg_price_cat(df, category):
    
    # Compute the average prices
    avg_prices = purchases.loc[purchases['category_id'] == category].groupby('brand').mean()['price']
    
    # Plot them
    f = plt.figure()
    ax = avg_prices.plot(figsize=(15,6), kind='bar', color = "royalblue", zorder=3)

    # Set up grids
    plt.grid(color = 'lightgray', linestyle='-.', zorder = 0)

    # setting label for x, y and the title
    plt.setp(ax,xlabel='brands', ylabel='avg price',
             title = 'Average price for brand')
    
    plt.show()
    
    return

In [ ]:
# e.g., if `category = 2053013555631882655

avg_price_cat(df_Nov, 2053013555631882655)

### 3.b
Find, for each category, the brand with the highest average price. Return all the results in ascending order by price.

In [ ]:
def highest_price_brands(df):
    # Create a dataframe with just the purchases as event_type
    df_purchases = df[df.event_type == 'purchase']
    
    # Select only the ones where the `brand` column is not empty
    df_notnull_purchases = df_purchases.loc[df_purchases.brand.notnull()]
    
    # Instantiate a dictionary
    high_brands = {}
    
    # Fill with the category number as key and its (brand, price) as values
    # brand selected will be the one with avg highest price in the selected category
    # Iterate on the dframes created by the groupby on the category
    for _, category_frame in df_notnull_purchases.groupby('category_id'):
        category_num = category_frame['category_id'].iloc[0]
        
        # For each frame, group on the brand and transform with mean
        avg_prices_cat = category_frame.groupby('brand').mean().reset_index()
        
        # Select row index with highest price
        idx = avg_prices_cat['price'].argmax()
        
        # Extract brand name and respective price for each category
        category_brand = avg_prices_cat.iloc[idx]['brand']
        category_price = avg_prices_cat.iloc[idx]['price']
        
        # Fill the dictionary
        high_brands[category_num] = (category_brand, category_price)
        
    dict_values = list(high_brands.values())
    
    # Sort the dictionary values (brand, price) w.r.t. the price
    dict_values.sort(key=lambda x:x[1])
    
    # Return the keys, which are the actual brands sorted
    return [x[0] for x in dict_values]

In [ ]:
highest_price_brands(df_Nov)

## [RQ5]
In what part of the day is your store most visited? Knowing which days of the week or even which hours of the day shoppers are likely to visit your online store and make a purchase may help you improve your strategies. Create a plot that for each day of
the week show the hourly average of visitors your store has.

In [ ]:
def avg_users(df):
    
    week_days = []
    
    for _, week_day_df in df.groupby([df.event_time.dt.weekday]):
        users_num = week_day_df.groupby([week_day_df.event_time.dt.hour]).count()['user_id']
        week_days.append((users_num, week_day_df.event_time.iloc[0].strftime('%A')))
        
    plots_colors = ['royalblue', 'orange', 'mediumseagreen', 
                    'crimson', 'darkcyan', 'coral', 'violet']
    
    
    # For every day of the week, plot the average number of users that visit the store each hour
    for i, (week_day, day_name) in enumerate(week_days):
        # Plot them
        f = plt.figure()
        
        ax = week_day.plot(figsize=(15,6), kind='bar', color = plots_colors[i], zorder=3)
        
        # Set up grids
        plt.grid(color = 'lightgray', linestyle='-.', zorder = 0)

        # setting label for x, y and the title
        plt.setp(ax,xlabel='hour', ylabel='avg users',
                 title = 'Average number of users per hour - {}'.format(day_name))

        plt.show()
    
    return

In [ ]:
# Use sample for time reasons

avg_users(sample)

## [RQ6]
The conversion rate of a product is given by the purchase rate over the number of times the product has been visited. What's the conversion rate of your online store?

### 6.a
Find the overall conversion rate of your store.